In [ ]:
!python3 -m spacy download en_core_web_md

In [ ]:
import sys
sys.path.append('/mnt/bitgraph')
sys.path.append('/mnt/gremlin++')
from pybitgraph import BitGraph

In [ ]:
import pandas
df = pandas.read_parquet('/mnt/bitgraph/data/rag/articles.parquet')
df

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

In [ ]:
t1 = nlp('brown fox')

m = df.title.map(lambda x : nlp(x).vector)
m

In [ ]:
import numpy as np
emb = np.array(m.values.tolist())
emb

In [ ]:
ner = nlp.add_pipe('ner')

In [ ]:
df.title[:10]

In [ ]:
nlp.pipe('ner')('What is the capital of Afghanistan?')

In [ ]:
for doc in nlp.pipe(['What is the capital of Afghanistan?']):
    print(doc.ents[0].vector.shape)

In [ ]:
np.save('/mnt/bitgraph/data/rag/emb.npy', emb)

In [ ]:
import numpy as np
emb = np.load('/mnt/bitgraph/data/rag/emb.npy')

In [ ]:
import cupy
cupy_emb = cupy.array(emb)
cupy_emb.shape

In [ ]:
import cudf
s = cudf.DataFrame(cupy_emb)
s

In [ ]:
from cuml.metrics.pairwise_distances import pairwise_distances
v = cupy.array(nlp('Afghanistan').vector).reshape((1,300))

matches = pairwise_distances(
    s,
    v
).sort_values()[:10]
matches

In [ ]:
import pandas
adf = pandas.read_parquet('/mnt/bitgraph/data/rag/articles.parquet')
edf = pandas.read_parquet('/mnt/bitgraph/data/rag/edgelist.parquet')

In [ ]:
adf.iloc[matches.index.to_numpy()]

In [ ]:
vertex_ids = adf.iloc[matches.index.to_numpy()].new_id.tolist()
vertex_ids

In [ ]:
#vertex_ids = [213280, 30617, 91, 103081, 151051]
vertex_ids = [213280, 30617, 91, 103081, 151051, 4586093, 1098, 5844157, 4107202, 5291135]

In [ ]:
num_vertices = max(edf.src.max(), edf.dst.max(), len(adf))
num_vertices

In [ ]:
from pybitgraph import BitGraph

graph = BitGraph(
    "int64",
    "int64",
    "DEVICE",
    "MANAGED",
    "MANAGED",
)

graph.add_vertices(num_vertices)
graph.add_edges(
    edf.src.values,
    edf.dst.values,
    'link'
)

g = graph.traversal()

In [ ]:
from pygremlinxx import GraphTraversal
__ = lambda : GraphTraversal()

from time import perf_counter

for _ in range(11):
    start_time = perf_counter()
    additional_docs = g.V(vertex_ids).emit(__().identity()).repeat(__().out().dedup().order().by(__().out().count()).limit(10)).times(2).dedup().order().by(__().out().count()).toArray()
    end_time = perf_counter()
    print(end_time - start_time)
additional_docs

In [ ]:
from pygremlinxx import TraversalStrategy
from pygremlinxx import GraphTraversal
__ = lambda : GraphTraversal()

from time import perf_counter

h = graph.traversal().withoutStrategy(BitGraph.BitGraphSelectionStrategy).withoutStrategy(TraversalStrategy.BasicPatternExtractionStrategy).withoutStrategy(TraversalStrategy.RepeatUnrollStrategy).withoutStrategy(TraversalStrategy.HasJoinStrategy)

for _ in range(11):
    start_time = perf_counter()
    additional_docs = h.V(vertex_ids).emit(__().identity()).repeat(__().out().dedup().order().by(__().out().count()).limit(10)).times(2).dedup().order().by(__().out().count()).toArray()
    end_time = perf_counter()
    print(end_time - start_time)
additional_docs

In [ ]:
edf

In [ ]:
from pygremlinxx import GraphTraversal
__ = lambda : GraphTraversal()
print(g.V([1,2,3]).emit(__().identity()).repeat(__().out().dedup().order().by(__().out().count()).limit(10)).times(2).dedup().order().by(__().out().count()).explain())